In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.model_zoo as model_zoo
import math
import numpy as np
import time

from cnn3d.model3d import Cnn3d
from cnn3d.data import get_training_set, get_test_set


In [4]:
#DATA_DIR = '/a/h/atong01/input/'
DATA_DIR = '/a/data/lungdl/'
lungs_dir = DATA_DIR + '3Darrays_visual/'
labels_file = DATA_DIR + 'stage1_labels.csv'
batch_size = 1

trainset = get_training_set(lungs_dir, labels_file)
testset = get_test_set(lungs_dir, labels_file)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)#, num_workers=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)#, num_workers=1)

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import shutil
def imshow(img):
    npimg = img.numpy()
    plt.imshow(npimg)
    plt.show()
    
dataiter = iter(trainloader)
count = 0
total = 0
good_set = set()
for i,(images, label,f) in enumerate(dataiter):
#images, labels = dataiter.next()
    
    for img in images:
        middle_slice = img[0][30]
        bottom_right = img[0,30,223,223]
        total += 1
        #print(i,img[0,30,223,223], if bottom_right==-10000)
        if bottom_right !=-10000:
            #print i,f
            count += 1
            #imshow(middle_slice)
        else:
            good_set.add(f)
        #print(img.size())
print ("Percent bad",float(count) / total)
# good_set
for f in good_set:
    shutil.copy(lungs_dir + f[0], DATA_DIR + '3D_pruned/' + f[0])

#print(' '.join('{label}'.format(label=labels[j]) for j in range(batch_size)))

('Percent bad', 0.113)


In [11]:
print(len(good_set))
print total

351
397


In [4]:
 for label in labels: 
    print (labels)



(0 ,.,.) = 
  0
[torch.FloatTensor of size 1x1x1]



In [5]:
from cnn3d.model3d import Cnn3d
net = Cnn3d().cuda()
i, data = next(enumerate(trainloader, 0))
inputs, labels = data
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
outputs = net.features(inputs)
print(outputs.size())

print(net(inputs))

torch.Size([1, 16, 7, 7, 7])


RuntimeError: CUDNN_STATUS_BAD_PARAM

In [ ]:
def train_model(model,dset_loaders, criterion, optimizer, lr_scheduler, num_epochs=25):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if torch.cuda.is_available():
                    inputs, labels = Variable(inputs.cuda()), \
                        Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model

def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [ ]:

if torch.cuda.is_available():
    net = net.cuda()

criterion = nn.BCELoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
optimizer_ft = optim.Adam(net.parameters())



In [7]:
dset_loaders = { 'train': trainloader, 'val': testloader}
model_ft = train_model(net,dset_loaders, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

In [ ]:
def evaluate_model(model, dset_loader):
    model.train(False)  # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    preds = torch.ByteTensor(len(dset_loader))
    all_labels = torch.ByteTensor(len(dset_loader))

    # Iterate over data.
    for i, data in enumerate(dset_loader):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs, labels = Variable(inputs.cuda()), \
                Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # forward
        outputs = model(inputs)
        #_, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        # statistics
        running_loss += loss.data[0]
        running_corrects += torch.sum((outputs.data > .5) == (labels.data > .5))
        #collect preds and all_labels
        all_labels[i] = (labels.data > .5)[0,0,0]
        preds[i] = (outputs.data > .5)[0,0]

    epoch_loss = running_loss / len(dset_loader)
    epoch_acc = running_corrects / len(dset_loader)

    print('Loss: {:.4f} Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))
    print()

    return (all_labels, preds)

In [ ]:
all_labels, preds = evaluate_model(model_ft, dset_loaders['val'])

In [ ]:
all_labels = all_labels.numpy()
preds = preds.numpy()

print (all_labels, preds)